In [1]:
# These two lines are necessary only if GemPy is not installed via pip
import sys, os
sys.path.append(r'../../gempy/')
#sys.path.append(r'/Users/nijan/Documents/git/gempy/')

# Importing GemPy
import gempy as gp
from gempy.assets import topology as tp
from gempy.bayesian.fields import compute_prob, calculate_ie_masked

# Importing auxilary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# For embedding matplotlib figures
#%matplotlib qt5
%matplotlib inline

# Import improved plotting features
from gempy.plot import visualization_2d as vv
from gempy.plot import vista

import seaborn as sn

In [2]:
## progress bar
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

# Model Initialization
We import the base model with its topography etc.

In [3]:
model_path = '../models/20210319_MC_no_middle_filling/'

geo_model = gp.load_model('Graben_base_model', path=model_path,
                         recompile=False)

Active grids: ['regular']
Active grids: ['regular' 'topography']


In [4]:
geo_model.get_additional_data()

values
Structure isLith                                                          True
          isFault                                                         True
          number faults                                                      5
          number surfaces                                                   11
          number series                                                     10
          number surfaces per series            [1, 1, 1, 1, 1, 3, 1, 1, 1, 0]
          len surfaces surface_points  [6, 8, 12, 8, 6, 8, 14, 14, 26, 24, 24]
          len series surface_points        [6, 8, 12, 8, 6, 36, 26, 24, 24, 0]
          len series orientations           [2, 2, 6, 4, 2, 12, 16, 12, 10, 0]
Options   dtype                                                        float64
          output                                                       geology
          theano_optimizer                                        fast_compile
          device                                                           cpu
          verbosity                                                       None
Kriging   range                                                        32190.8
          $C_o$                                                    2.46726e+07
          drift equations                       [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
Rescaling rescaling factor                                             56916.7
          centers                          [14239.166495, 6900.0, -2581.92853]

In [5]:
# adapt kriging to the parameters of previous example
# decrease the kriging range
geo_model.modify_kriging_parameters('range', 20000.)
geo_model.modify_kriging_parameters('$C_o$', 2e5)
#geo_model.modify_surface_points('all', smooth=1e-6)

# Set the interpolator function
# Create the theano model
gp.set_interpolator(geo_model,
                         compile_theano=True,
                         theano_optimizer='fast_compile',
                         verbose=[],
                         update_kriging=False);

Compiling theano function...
Level of Optimization:  fast_compile
Device:  cpu
Precision:  float64
Number of faults:  5
Compilation Done!
Kriging values: 
                                          values
range                                     20000
$C_o$                                    200000
drift equations  [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [6]:
# Compute the model
sol = gp.compute_model(geo_model, compute_mesh=True)

C:\Users\brigg\miniconda3\envs\env_gempy\lib\site-packages\gempy\core\solution.py:315: UserWarning: Surfaces not computed due to: No surface found at the given iso value.. The surface is: Series: No surface found at the given iso value.; Surface Number:8
  '; Surface Number:' + str(s_n))
C:\Users\brigg\miniconda3\envs\env_gempy\lib\site-packages\gempy\core\solution.py:128: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  self.geological_map = np.array([values[0][:, l0: l1], values[4][:, l0: l1].astype(float)])


## MC Variation

In [7]:
depth_zero = geo_model.surface_points.df['Z'].copy()
depth = depth_zero.copy()

In [8]:
graben_bot = geo_model.surface_points.df.query("surface=='Lower-filling' or surface=='Upper-filling'")

In [9]:
# mask_surfpoints = geo_model.surface_points.df.series.isin(faults)
np.random.seed(420)
lith_blocks = np.array([])
edge_blocks = dict()
cent_blocks = dict()
depth = geo_model.surface_points.df['Z'].copy()
graben = list(graben_bot.index)
n_points = len(graben)
n_iterations = 100

geo_model._grid
#for i in range(n_iterations):
for i in log_progress(range(n_iterations), name='Models'):
    variation = np.random.normal(0, 100)
    Z = depth[graben] + np.random.normal(variation, 50, size=n_points)
    geo_model.modify_surface_points(graben, Z=Z)
    gp.compute_model(geo_model)
    edges, centroids = tp.compute_topology(geo_model)
    lith_blocks = np.append(lith_blocks, geo_model.solutions.lith_block)
    edge_blocks[f"Real_{i}"] = edges
    cent_blocks[f"Real_{i}"] = centroids

lith_blocks = lith_blocks.reshape(n_iterations, -1)

C:\Users\brigg\miniconda3\envs\env_gempy\lib\site-packages\gempy\core\solution.py:315: UserWarning: Surfaces not computed due to: No surface found at the given iso value.. The surface is: Series: No surface found at the given iso value.; Surface Number:8
  '; Surface Number:' + str(s_n))
C:\Users\brigg\miniconda3\envs\env_gempy\lib\site-packages\gempy\core\solution.py:128: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  self.geological_map = np.array([values[0][:, l0: l1], values[4][:, l0: l1].astype(float)])


In [10]:
np.savetxt('../models/20210319_MC_no_middle_filling/lith_block_samples', lith_blocks, fmt='%d')

In [19]:
dist = np.array([])
for i in range(10000):
    dist = np.append(dist, np.random.normal(0, 100))

sn.distplot(dist)